In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
from tqdm import tqdm
from itertools import chain

import re  

In [2]:
import os
file_list = os.listdir('뉴스기사 저장파일 경로')
file_list


['news_search_result_111퍼센트.csv',
 'news_search_result_215맨션.csv',
 'news_search_result_21세기.csv',
 'news_search_result_21세기종합건설.csv',
 'news_search_result_3지스톤.csv',
 'news_search_result_88콘크리트.csv',
 'news_search_result_가나개발.csv',
 'news_search_result_가나마린.csv',
 'news_search_result_가나안.csv',
 'news_search_result_가나안전자정밀.csv',
 'news_search_result_가나에너지.csv',
 'news_search_result_가나엔텍.csv',
 'news_search_result_가나종합.csv',
 'news_search_result_가나종합철강.csv',
 'news_search_result_가나축산물유통.csv',
 'news_search_result_가나테크.csv',
 'news_search_result_가나테크화성지점.csv',
 'news_search_result_가노수.csv',
 'news_search_result_가누다.csv',
 'news_search_result_가드너.csv',
 'news_search_result_가드넥.csv',
 'news_search_result_가람에스티.csv',
 'news_search_result_가람이앤씨(광주지점).csv',
 'news_search_result_가랑해운점.csv',
 'news_search_result_가로.csv',
 'news_search_result_가로커뮤니케이션.csv',
 'news_search_result_가리미.csv',
 'news_search_result_가림종합건설.csv',
 'news_search_result_가미.csv',
 'news_search_result_가보.csv',
 'news_search_r

In [24]:
from tqdm import tqdm
os.chdir('뉴스기사 저장파일 경로')

dfs = [pd.read_csv(file) for file in tqdm(file_list[:4754])]    # 담당 범위 인덱스
combined_df = pd.concat(dfs, axis=0)

100%|██████████| 4754/4754 [00:15<00:00, 313.40it/s]


In [25]:
combined_df = combined_df.reset_index(drop=True)
combined_df

,title,originallink,link,description,pubDate,company,contents
0,NHN클라우드 게임 특화 클라우드로 미·일 공략,http://www.smedaily.co.kr/news/articleView.htm...,http://www.smedaily.co.kr/news/articleView.htm...,"현재 그라비티, 111퍼센트, 로드컴플릿, 위메이드 커넥트, 클로바게임즈 등 다양한...","Thu, 22 Jun 2023 15:40:00 +0900",111퍼센트,[]
1,"NHN클라우드, 준인터 신작 MMORPG ‘허츠워’에 클라우드 인프라 제공",https://biz.chosun.com/it-science/ict/2023/06/...,https://n.news.naver.com/mnews/article/366/000...,"현재 엔씨소프트, 위메이드, 클로버게임즈, 111퍼센트 등 다양한 게임사가 NHN클...","Fri, 16 Jun 2023 09:09:00 +0900",111퍼센트,['[게임 허츠워 포스터./ 준인터 제공 NHN클라우드는 게임사 준인터...
2,"111퍼센트, 슈퍼드리머 3기 참가자 접수 마감...전년比 지원율 160%↑",https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,사진=111퍼센트 111퍼센트는 자사의 인재 육성 인턴십 프로그램 슈퍼드리머의 3...,"Fri, 16 Jun 2023 08:26:00 +0900",111퍼센트,[]
3,"111퍼센트, ‘슈퍼드리머’ 3기 참가자 전년비 160%",https://byline.network/?p=9004111222506982,https://byline.network/?p=9004111222506982,유명한 111퍼센트(대표 김강안)는 자사의 인재 육성 인턴십 프로그램 슈퍼드리머의 ...,"Thu, 15 Jun 2023 17:48:00 +0900",111퍼센트,[]
4,[6월 15일] 승리의 여신: 니케 첫 여름맞이 이벤트 개최 등 게임업계 핫클립,http://www.gameple.co.kr/news/articleView.html...,http://www.gameple.co.kr/news/articleView.html...,111퍼센트 - 자사의 인재 육성 인턴십 프로그램 슈퍼드리머의 3기 참가자 접수를 ...,"Thu, 15 Jun 2023 17:44:00 +0900",111퍼센트,[]
...,...,...,...,...,...,...,...
382221,“핵심은 근거 기반”…현명한 선택 확산할까?,http://www.yakup.com/news/index.html?mode=view...,http://www.yakup.com/news/index.html?mode=view...,의료계가 과잉 진료 문제 해결을 위한 ‘현명한 선택(Choosing Wisely) ...,"Fri, 16 Dec 2022 06:02:00 +0900",삼성의료고무,[]
382222,[이슈대담] 농협금융 회장에 이석준 전 국무조정실장 낙점...관치 낙하산 신호...,https://www.nbntv.kr/news/articleView.html?idx...,https://www.nbntv.kr/news/articleView.html?idx...,그리고 윤석열 대통령의 대선 캠프 초기에 좌장을 맡기도 했고 특별고문으로 활동한 친...,"Wed, 14 Dec 2022 17:42:00 +0900",삼성의료고무,[]
382223,"첨단 생태계 키운 말레이, 이차전지 등 핵심 공급망 부상 [2022 세계아세안포...",http://www.segye.com/content/html/2022/12/13/2...,https://n.news.naver.com/mnews/article/022/000...,"말레이시아는 고무, 팜유 등 전통적인 산업에서 첨단 산업으로의 도약을 꾀하고 있다....","Tue, 13 Dec 2022 21:17:00 +0900",삼성의료고무,"[""[세션 1 - 투자 각광받는 교역 환경팜유·구리 등 25개 제품 최대 수출서반도..."
382224,"오스템임플란트, 수출 2억불 돌파…내년 수출 더 밝다",https://www.ekn.kr/web/view.php?key=2022121201...,https://www.ekn.kr/web/view.php?key=2022121201...,오스템임플란트는 지난 5일 서울 삼성동 코엑스에서 열린 ‘제59회 무역의 날’ 기념...,"Tue, 13 Dec 2022 06:32:00 +0900",삼성의료고무,[]


In [26]:
# 회사명 3글자 이상인 회사만 추출
combined_df = combined_df[combined_df['company'].str.len() >= 3]

In [27]:
# 유니크한 회사명을 저장
comp_list = combined_df['company'].unique()

In [33]:
combined_df['contents'] = combined_df['contents'].str.strip("[]").replace('', np.nan)
combined_df['contents']

0                                                       NaN
1         '[게임 허츠워 포스터./ 준인터 제공        NHN클라우드는 게임사 준인터의...
2                                                       NaN
3                                                       NaN
4                                                       NaN
                                ...                        
342953                                                  NaN
342954                                                  NaN
342955    "[세션 1 - 투자 각광받는 교역 환경팜유·구리 등 25개 제품 최대 수출서반도체...
342956                                                  NaN
342957                                                  NaN
Name: contents, Length: 342958, dtype: object

In [29]:
# 불필요한 회사명 제거
combined_df.iloc[np.where(combined_df.company == '21세기')[0]]
combined_df = combined_df[combined_df['company'] != '21세기']
combined_df = combined_df[combined_df['company'] != '메리트']
combined_df = combined_df[combined_df['company'] != '과일장터']
combined_df = combined_df[combined_df['company'] != '그린에너지']
combined_df = combined_df[combined_df['company'] != '그린에너지개발']
combined_df = combined_df[combined_df['company'] != '그린식품']
combined_df = combined_df[combined_df['company'] != '그린시티']
combined_df = combined_df[combined_df['company'] != '그린에코']
combined_df = combined_df[combined_df['company'] != '글로벌']
combined_df = combined_df[combined_df['company'] != '글로리아']
combined_df = combined_df.reset_index(drop=True)
combined_df

,title,originallink,link,description,pubDate,company,contents
0,NHN클라우드 게임 특화 클라우드로 미·일 공략,http://www.smedaily.co.kr/news/articleView.htm...,http://www.smedaily.co.kr/news/articleView.htm...,"현재 그라비티, 111퍼센트, 로드컴플릿, 위메이드 커넥트, 클로바게임즈 등 다양한...","Thu, 22 Jun 2023 15:40:00 +0900",111퍼센트,NaN
1,"NHN클라우드, 준인터 신작 MMORPG ‘허츠워’에 클라우드 인프라 제공",https://biz.chosun.com/it-science/ict/2023/06/...,https://n.news.naver.com/mnews/article/366/000...,"현재 엔씨소프트, 위메이드, 클로버게임즈, 111퍼센트 등 다양한 게임사가 NHN클...","Fri, 16 Jun 2023 09:09:00 +0900",111퍼센트,'[게임 허츠워 포스터./ 준인터 제공 NHN클라우드는 게임사 준인터의...
2,"111퍼센트, 슈퍼드리머 3기 참가자 접수 마감...전년比 지원율 160%↑",https://www.pinpointnews.co.kr/news/articleVie...,https://www.pinpointnews.co.kr/news/articleVie...,사진=111퍼센트 111퍼센트는 자사의 인재 육성 인턴십 프로그램 슈퍼드리머의 3...,"Fri, 16 Jun 2023 08:26:00 +0900",111퍼센트,NaN
3,"111퍼센트, ‘슈퍼드리머’ 3기 참가자 전년비 160%",https://byline.network/?p=9004111222506982,https://byline.network/?p=9004111222506982,유명한 111퍼센트(대표 김강안)는 자사의 인재 육성 인턴십 프로그램 슈퍼드리머의 ...,"Thu, 15 Jun 2023 17:48:00 +0900",111퍼센트,NaN
4,[6월 15일] 승리의 여신: 니케 첫 여름맞이 이벤트 개최 등 게임업계 핫클립,http://www.gameple.co.kr/news/articleView.html...,http://www.gameple.co.kr/news/articleView.html...,111퍼센트 - 자사의 인재 육성 인턴십 프로그램 슈퍼드리머의 3기 참가자 접수를 ...,"Thu, 15 Jun 2023 17:44:00 +0900",111퍼센트,NaN
...,...,...,...,...,...,...,...
342953,“핵심은 근거 기반”…현명한 선택 확산할까?,http://www.yakup.com/news/index.html?mode=view...,http://www.yakup.com/news/index.html?mode=view...,의료계가 과잉 진료 문제 해결을 위한 ‘현명한 선택(Choosing Wisely) ...,"Fri, 16 Dec 2022 06:02:00 +0900",삼성의료고무,NaN
342954,[이슈대담] 농협금융 회장에 이석준 전 국무조정실장 낙점...관치 낙하산 신호...,https://www.nbntv.kr/news/articleView.html?idx...,https://www.nbntv.kr/news/articleView.html?idx...,그리고 윤석열 대통령의 대선 캠프 초기에 좌장을 맡기도 했고 특별고문으로 활동한 친...,"Wed, 14 Dec 2022 17:42:00 +0900",삼성의료고무,NaN
342955,"첨단 생태계 키운 말레이, 이차전지 등 핵심 공급망 부상 [2022 세계아세안포...",http://www.segye.com/content/html/2022/12/13/2...,https://n.news.naver.com/mnews/article/022/000...,"말레이시아는 고무, 팜유 등 전통적인 산업에서 첨단 산업으로의 도약을 꾀하고 있다....","Tue, 13 Dec 2022 21:17:00 +0900",삼성의료고무,"""[세션 1 - 투자 각광받는 교역 환경팜유·구리 등 25개 제품 최대 수출서반도체..."
342956,"오스템임플란트, 수출 2억불 돌파…내년 수출 더 밝다",https://www.ekn.kr/web/view.php?key=2022121201...,https://www.ekn.kr/web/view.php?key=2022121201...,오스템임플란트는 지난 5일 서울 삼성동 코엑스에서 열린 ‘제59회 무역의 날’ 기념...,"Tue, 13 Dec 2022 06:32:00 +0900",삼성의료고무,NaN


In [36]:
combined_df = combined_df.dropna(subset=['contents'], axis=0)
combined_df = combined_df.reset_index(drop=True)
combined_df

,title,originallink,link,description,pubDate,company,contents
0,"NHN클라우드, 준인터 신작 MMORPG ‘허츠워’에 클라우드 인프라 제공",https://biz.chosun.com/it-science/ict/2023/06/...,https://n.news.naver.com/mnews/article/366/000...,"현재 엔씨소프트, 위메이드, 클로버게임즈, 111퍼센트 등 다양한 게임사가 NHN클...","Fri, 16 Jun 2023 09:09:00 +0900",111퍼센트,'[게임 허츠워 포스터./ 준인터 제공 NHN클라우드는 게임사 준인터의...
1,"111퍼센트, 인재 육성 인턴십 프로그램 슈퍼드리머 3기 접수 마감",https://zdnet.co.kr/view/?no=20230615144250,https://n.news.naver.com/mnews/article/092/000...,111퍼센트(대표 김강안)는 자사의 인재 육성 인턴십 프로그램 슈퍼드리머의 3기 참...,"Thu, 15 Jun 2023 14:51:00 +0900",111퍼센트,"""[\t\t\t111퍼센트(대표 김강안)는 자사의 인재 육성 인턴십 프로그램 '슈퍼..."
2,"111퍼센트 “인턴십 지원, 260여명 몰려…전 기수 대비 160% 증가”",https://www.ddaily.co.kr/page/view/20230615110...,https://n.news.naver.com/mnews/article/138/000...,111퍼센트(대표 김강안)는 인재 육성 인턴십 프로그램 ‘슈퍼드리머’ 3기 참가자 ...,"Thu, 15 Jun 2023 13:50:00 +0900",111퍼센트,'[[사진=111퍼센트][디지털데일리 왕진화 기자] 111퍼센트(대표 김강안)는 인...
3,"111퍼센트, 인턴십 프로그램 슈퍼드리머 모집... 전년比 지원율 160%↑",https://www.xportsnews.com/article/1736194,https://sports.news.naver.com/news.nhn?oid=311...,111퍼센트가 올해로 3회째를 맞는 인재 육성 인턴십 프로그램 슈퍼드리머를 모집했다...,"Thu, 15 Jun 2023 13:30:00 +0900",111퍼센트,'[]'
4,"111퍼센트, 인턴십 모집 마감…전년비 지원율 160% 증가",http://www.newsis.com/view/?id=NISX20230615_00...,https://n.news.naver.com/mnews/article/003/001...,111퍼센트는 인재 육성 인턴십 프로그램 슈퍼드리머의 3기 참가자 접수를 성황리에 ...,"Thu, 15 Jun 2023 10:14:00 +0900",111퍼센트,"'[인재 육성 인턴십 프로그램 \'슈퍼드리머\' 3기 참가자 접수""260여명 역대급..."
...,...,...,...,...,...,...,...
116567,"시장분석업체 새해 IT기술 전망, 3대 키워드는",https://zdnet.co.kr/view/?no=20221227114847,https://n.news.naver.com/mnews/article/092/000...,"최근 애널리스트 프레젠테이션, 삼성전자, EE, 퀄컴 등의 경영진과 인터뷰 및 10...","Tue, 27 Dec 2022 13:21:00 +0900",삼성의료고무,'[\t\t\t매년 연말이면 여러 시장조사업체가 새해의 IT 시장전망을 내놓는다. ...
116568,인재경영 완성은 근로자 건강관리…건강친화기업 첫 인증,https://www.mk.co.kr/article/10575684,https://n.news.naver.com/mnews/article/009/000...,"⑫동일고무벨트 ⑬삼성전자 DS부문 ⑭모노랩스 등 14개사 인증받아 복지부, 2022...","Tue, 20 Dec 2022 17:54:00 +0900",삼성의료고무,'[총 41개사 신청해 ①한국천문연구원 ②에스포항병원 ③한화생명보험 ④현대일렉트릭앤...
116569,[ET] ‘산타’아닌 ‘사탄’온다? “내년 코스피는 달러와 중국에 달렸다”,https://news.kbs.co.kr/news/view.do?ncd=572288...,https://n.news.naver.com/mnews/article/056/001...,특히 고무적인 부분은 장기성 자금들이 들어오고 있는 것이 관측이 된다는 점입니다. ...,"Tue, 20 Dec 2022 17:52:00 +0900",삼성의료고무,'[■ 프로그램명 : 통합뉴스룸ET■ 코너명 : ET WHY?■ 방송시간 : 12월...
116570,"삼성전자·한화생명·기아·국민은행, 근로자 ‘건강친화인증기업’ 선정",https://biz.chosun.com/topics/topics_social/20...,https://n.news.naver.com/mnews/article/366/000...,"동일고무벨트, 삼성전자 DS부문, 모노랩스 등이다. 총 41개사가 신청했고, 14개...","Tue, 20 Dec 2022 12:44:00 +0900",삼성의료고무,"'[동일고무, 뉴로피드백 훈련 프로그램으로야간 근무자 수면 효과 개선 ..."


In [37]:
pattern = r'[\\t\\t\\t\n■]'  # 제거하고자 하는 특수 문자 패턴


combined_df['contents'] = combined_df['contents'].apply(lambda x: re.sub(pattern, '', str(x)))
combined_df['contents']

0         '[게임 허츠워 포스터./ 준인터 제공        NHN클라우드는 게임사 준인터의...
1         "[111퍼센트(대표 김강안)는 자사의 인재 육성 인턴십 프로그램 '슈퍼드리머'의 ...
2         '[[사진=111퍼센트][디지털데일리 왕진화 기자] 111퍼센트(대표 김강안)는 인...
3                                                      '[]'
4         '[인재 육성 인턴십 프로그램 '슈퍼드리머' 3기 참가자 접수"260여명 역대급 지...
                                ...                        
116567    '[매년 연말이면 여러 시장조사업체가 새해의 IT 시장전망을 내놓는다. 최근 미국지...
116568    '[총 41개사 신청해 ①한국천문연구원 ②에스포항병원 ③한화생명보험 ④현대일렉트릭앤...
116569    '[ 프로그램명 : 통합뉴스룸ET 코너명 : ET WHY? 방송시간 : 12월20일...
116570    '[동일고무, 뉴로피드백 훈련 프로그램으로야간 근무자 수면 효과 개선        ...
116571    "[세션 1 - 투자 각광받는 교역 환경팜유·구리 등 25개 제품 최대 수출서반도체...
Name: contents, Length: 116572, dtype: object

In [106]:
# output_df  = pd.DataFrame()
idx = []
le = 0
for i in tqdm(comp_list):
    rows = np.where(combined_df.loc[np.where(combined_df['company'] == i)[0], 'contents'].str.contains(i))[0]
    rows = rows.tolist()
    result = [num + (le) for num in rows]
    idx.append(result)
    le += len(np.where(combined_df['company'] == i)[0])


  0%|          | 13/4360 [00:00<00:34, 127.29it/s]<ipython-input-106-c1b6e775847f>:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  rows = np.where(combined_df.loc[np.where(combined_df['company'] == i)[0], 'contents'].str.contains(i))[0]
  1%|          | 27/4360 [00:00<00:32, 132.39it/s]<ipython-input-106-c1b6e775847f>:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  rows = np.where(combined_df.loc[np.where(combined_df['company'] == i)[0], 'contents'].str.contains(i))[0]
  5%|▍         | 209/4360 [00:01<00:31, 133.89it/s]<ipython-input-106-c1b6e775847f>:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  rows = np.where(combined_df.loc[np.where(combined_df['company'] == i)[0], 'contents'].str.contains(i))[0]
  6%|▋         | 2

In [107]:
ids = list(chain.from_iterable(idx))
filtered_df = combined_df.iloc[ids]

In [108]:
filtered_df = filtered_df.reset_index(drop=True)
filtered_df

,title,originallink,link,description,pubDate,company,contents
0,"NHN클라우드, 준인터 신작 MMORPG ‘허츠워’에 클라우드 인프라 제공",https://biz.chosun.com/it-science/ict/2023/06/...,https://n.news.naver.com/mnews/article/366/000...,"현재 엔씨소프트, 위메이드, 클로버게임즈, 111퍼센트 등 다양한 게임사가 NHN클...","Fri, 16 Jun 2023 09:09:00 +0900",111퍼센트,'[게임 허츠워 포스터./ 준인터 제공 NHN클라우드는 게임사 준인터의...
1,"111퍼센트, 인재 육성 인턴십 프로그램 슈퍼드리머 3기 접수 마감",https://zdnet.co.kr/view/?no=20230615144250,https://n.news.naver.com/mnews/article/092/000...,111퍼센트(대표 김강안)는 자사의 인재 육성 인턴십 프로그램 슈퍼드리머의 3기 참...,"Thu, 15 Jun 2023 14:51:00 +0900",111퍼센트,"""[111퍼센트(대표 김강안)는 자사의 인재 육성 인턴십 프로그램 '슈퍼드리머'의 ..."
2,"111퍼센트 “인턴십 지원, 260여명 몰려…전 기수 대비 160% 증가”",https://www.ddaily.co.kr/page/view/20230615110...,https://n.news.naver.com/mnews/article/138/000...,111퍼센트(대표 김강안)는 인재 육성 인턴십 프로그램 ‘슈퍼드리머’ 3기 참가자 ...,"Thu, 15 Jun 2023 13:50:00 +0900",111퍼센트,'[[사진=111퍼센트][디지털데일리 왕진화 기자] 111퍼센트(대표 김강안)는 인...
3,"111퍼센트, 인턴십 모집 마감…전년비 지원율 160% 증가",http://www.newsis.com/view/?id=NISX20230615_00...,https://n.news.naver.com/mnews/article/003/001...,111퍼센트는 인재 육성 인턴십 프로그램 슈퍼드리머의 3기 참가자 접수를 성황리에 ...,"Thu, 15 Jun 2023 10:14:00 +0900",111퍼센트,"'[인재 육성 인턴십 프로그램 '슈퍼드리머' 3기 참가자 접수""260여명 역대급 지..."
4,"111퍼센트, 인턴십 3기 모집…지원자 160% 늘어",http://www.edaily.co.kr/news/newspath.asp?news...,https://n.news.naver.com/mnews/article/018/000...,111퍼센트는 자사의 인재 육성 인턴십 프로그램 ‘슈퍼드리머’의 3기 참가자 접수를...,"Thu, 15 Jun 2023 09:55:00 +0900",111퍼센트,'[[이데일리 김정유 기자] 111퍼센트는 자사의 인재 육성 인턴십 프로그램 ‘슈퍼...
...,...,...,...,...,...,...,...
29777,"[포토]강운태 광주시장,우수중소기업인상 수여",http://view.asiae.co.kr/news/view.htm?idxno=20...,https://n.news.naver.com/mnews/article/277/000...,"(왼쪽부터 백희종 (주)인아 대표, 이재수 (주)골드텔 대표, 안병열 삼성스텐레스상...","Fri, 27 Dec 2013 14:26:00 +0900",삼성스텐레스상공,'[[아시아경제 노해섭 기자]강운태 광주시장은 27일 오전 시청 비즈니스룸에서 (주...
29778,"광주시, 유망중소기업 13개사 선정",http://www.etnews.com/201108260066,https://n.news.naver.com/mnews/article/030/000...,"선정된 유망중소기업은 무등, 우리로광통신, 한국정밀, 이노셈코리아, 삼성스텐레스상공...","Fri, 26 Aug 2011 15:03:00 +0900",삼성스텐레스상공,'[u3000광주광역시는 향토기업 3곳을 포함해 13개 기업을 올해 유망 중소기업으...
29779,광주시 유망중소기업 13개 선정,NaN,https://n.news.naver.com/mnews/article/001/000...,다음은 선정기업. ▲무등 ▲이노셈코리아 ▲우리로광통신 ▲한국정밀 ▲삼성스텐레스상공 ...,"Thu, 25 Aug 2011 17:03:00 +0900",삼성스텐레스상공,'[ (광주=연합뉴스) 전승현 기자 = 광주시는 기술력이 우수하고 성장잠재력이...
29780,광주시 고용우수 인증기업 20개소 선정…인센티브,NaN,https://n.news.naver.com/mnews/article/003/000...,"이번에 선정된 기업은 지오메디칼을 비롯해 선호기업 주식회사, 한국IMC㈜, ㈜무등기...","Mon, 01 Aug 2011 17:31:00 +0900",삼성스텐레스상공,'[【광주=뉴시스】구길용 기자 = 광주시는 1일 고용창출에 기여한 20개 기업을 고...


In [115]:
# 경로 재설정
os.chdir('원래 경로')

In [116]:
# 중간에 한번 저장 (혹시 몰라서... 백업용, 생략해도 무방할듯)
filtered_df.to_csv('뉴스전처리_0702.csv', index=False)

In [117]:
# 문장 분리 함수
def split_sentences(row):
    sentences = row['contents'].split('.')
    return pd.DataFrame({'company': [row['company']] * len(sentences), 'contents': sentences})



In [118]:
# 문장 분리 적용
new_df = pd.concat(filtered_df.apply(split_sentences, axis=1).tolist(), ignore_index=True)

# 결과 출력
print(new_df)

         company                                           contents
0         111퍼센트                                       '[게임 허츠워 포스터
1         111퍼센트  / 준인터 제공        NHN클라우드는 게임사 준인터의 신규 MMORPG ‘허...
2         111퍼센트  허츠워는 ‘잡 체인지xa0RPG’라는 컨셉에 맞춰 동종 장르 게임들과 달리 플레이어...
3         111퍼센트   15일 구글플레이 플랫폼에 정식 출시했으며,xa0근시일 내 앱스토어 플랫폼에도 출...
4         111퍼센트  NHN클라우드는 허츠워 운영에 필요한 서버,xa0스토리지,xa0네트워크 등의 인프라...
...          ...                                                ...
604754  삼성스텐레스상공   신상품 개발팀이 별도로 운영되고 있으며,올 하반기에는 한국 비즈니스맨의 의식개혁을...
604755  삼성스텐레스상공  최보영 상품개발 담당 책임연구원은 "자칫하면 무겁고 딱딱해질 수 있는 의식개혁 교육...
604756  삼성스텐레스상공  한무드는 '적당주의ㆍ괜찮아 병 타파' 강연회는 물론 다양한 교육 프로그램과 신상품을...
604757  삼성스텐레스상공                  (02)547-4405  김현지 기자 nuk@hankyung
604758  삼성스텐레스상공  comⓒ 한국경제신문, 무단 전재 및 재배포 금지&l;한국온라인신문협회의 디지털뉴스...

[604759 rows x 2 columns]


In [119]:
# 15자 미만인 기사의 행 삭제
new_df = new_df[new_df['contents'].str.len() >= 15]
# '@'를 포함한 행 삭제
new_df = new_df[~new_df['contents'].str.contains('@')]

new_df = new_df.reset_index(drop=True)
# 결과 출력
print(new_df)

         company                                           contents
0         111퍼센트  / 준인터 제공        NHN클라우드는 게임사 준인터의 신규 MMORPG ‘허...
1         111퍼센트  허츠워는 ‘잡 체인지xa0RPG’라는 컨셉에 맞춰 동종 장르 게임들과 달리 플레이어...
2         111퍼센트   15일 구글플레이 플랫폼에 정식 출시했으며,xa0근시일 내 앱스토어 플랫폼에도 출...
3         111퍼센트  NHN클라우드는 허츠워 운영에 필요한 서버,xa0스토리지,xa0네트워크 등의 인프라...
4         111퍼센트  xa0이에 준인터는 허츠워 서비스를 위한 자체 인프라 구축 시 발생할 수 있는 하드...
...          ...                                                ...
513452  삼성스텐레스상공       한무드는 이외 프로그램 개발을 위한 연구개발(R&amp;D)에도 힘을 쏟고 있다
513453  삼성스텐레스상공   신상품 개발팀이 별도로 운영되고 있으며,올 하반기에는 한국 비즈니스맨의 의식개혁을...
513454  삼성스텐레스상공  최보영 상품개발 담당 책임연구원은 "자칫하면 무겁고 딱딱해질 수 있는 의식개혁 교육...
513455  삼성스텐레스상공  한무드는 '적당주의ㆍ괜찮아 병 타파' 강연회는 물론 다양한 교육 프로그램과 신상품을...
513456  삼성스텐레스상공  comⓒ 한국경제신문, 무단 전재 및 재배포 금지&l;한국온라인신문협회의 디지털뉴스...

[513457 rows x 2 columns]


In [121]:
new_df.contents[10]

'"[111퍼센트(대표 김강안)는 자사의 인재 육성 인턴십 프로그램 \'슈퍼드리머\'의 3기 참가자 접수를 마감했다고 15일 밝혔다'

In [120]:
new_df.to_csv('기사전처리(~삼성의료고무)_0702.csv', index=False)